In [ ]:
# Imports
import utils
from utils import HELOC_NAME, COVERTYPE_NAME, CALIFORNIA_HOUSING_NAME, DENGUE_DATASET
import tensorflow as tf
import os
import pandas as pd
import cv2
import numpy as np
import concurrent.futures
from tensorflow.keras.utils import to_categorical
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
# Hyperparameters
DROPOUT_VALUES = [0.2, 0.5]

BATCH_SIZES = {
    HELOC_NAME: 64,
    COVERTYPE_NAME: 256,
    CALIFORNIA_HOUSING_NAME: 64, 
    DENGUE_DATASET: 64,
}
EPOCHS = 200

METRICS_CLASSIFICATION_BINARY = [
    tf.keras.metrics.Precision(),
    tf.keras.metrics.Recall(),
    tf.keras.metrics.BinaryAccuracy()
]

METRICS_CLASSIFICATION_MULTICLASS = [
    tf.keras.metrics.Precision(),
    tf.keras.metrics.Recall(),
    tf.keras.metrics.CategoricalAccuracy()
]

METRICS_REGRESSION = [
    tf.keras.metrics.MeanSquaredError(),
    tf.keras.metrics.MeanAbsoluteError()
]

MIDDLE_ACTIVATIONS = ["sigmoid", "relu"]

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, BatchNormalization, InputLayer, LayerNormalization
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Activation,MaxPooling2D, AveragePooling2D, Concatenate

def build_model(input_shape, is_classification_dataset, dropout_value, middle_activation, last_layer_n, n_classes=None):
    if is_classification_dataset:
        assert n_classes is not None
        assert n_classes > 1
    else:
        assert n_classes is None

    # Define the model architecture

    #Entrada
    input_shape = Input(input_shape)

    #Inicio de rama 1
    tower_1 = Conv2D(16, (3,3), activation='relu',padding="same")(input_shape)
    tower_1 = BatchNormalization()(tower_1)
    tower_1 = Activation('relu')(tower_1)
    tower_1 = MaxPooling2D(2,2)(tower_1)
    tower_1 = Dropout(dropout_value)(tower_1)

    tower_1 = Conv2D(32, (3,3), activation='relu',padding="same")(tower_1)
    tower_1 = BatchNormalization()(tower_1)
    tower_1 = Activation('relu')(tower_1)
    tower_1 = MaxPooling2D(2,2)(tower_1)
    tower_1 = Dropout(dropout_value)(tower_1)

    tower_1 = Conv2D(64, (3,3), activation='relu',padding="same")(tower_1)
    tower_1 = BatchNormalization()(tower_1)
    tower_1 = Activation('relu')(tower_1)
    tower_1 = MaxPooling2D(2,2)(tower_1)
    tower_1 = Dropout(dropout_value)(tower_1)

    tower_1 = Conv2D(last_layer_n, (3,3), activation='relu',padding="same")(tower_1)
    tower_1 = BatchNormalization()(tower_1)
    tower_1 = Activation('relu')(tower_1)
    tower_1 = MaxPooling2D(2,2)(tower_1)
    tower_1 = Dropout(dropout_value)(tower_1)
    #Fin de rama 1

    #Inicio de rama 2
    tower_2 = Conv2D(16, (5,5), activation='relu',padding="same")(input_shape)
    tower_2 = BatchNormalization()(tower_2)
    tower_2 = Activation('relu')(tower_2)
    tower_2 = AveragePooling2D(2,2)(tower_2)
    tower_2 = Dropout(dropout_value)(tower_2)

    tower_2 = Conv2D(32, (5,5), activation='relu',padding="same")(tower_2)
    tower_2 = BatchNormalization()(tower_2)
    tower_2 = Activation('relu')(tower_2)
    tower_2 = AveragePooling2D(2,2)(tower_2)
    tower_2 = Dropout(dropout_value)(tower_2)

    tower_2 = Conv2D(64, (5,5), activation='relu',padding="same")(tower_2)
    tower_2 = BatchNormalization()(tower_2)
    tower_2 = Activation('relu')(tower_2)
    tower_2 = AveragePooling2D(2,2)(tower_2)
    tower_2 = Dropout(dropout_value)(tower_2)

    tower_2 = Conv2D(last_layer_n, (5,5), activation='relu',padding="same")(tower_2)
    tower_2 = BatchNormalization()(tower_2)
    tower_2 = Activation('relu')(tower_2)
    tower_2 = AveragePooling2D(2,2)(tower_2)
    tower_2 = Dropout(dropout_value)(tower_2)
    #Fin de rama 2

    #Concatenación de las 2 ramas
    merged = Concatenate(axis=1)([tower_1, tower_2])

    #Aplanamiento
    merged = Flatten()(merged)

    #Capas adicionales
    out = Dense(256, activation='relu')(merged)
    out = Dense(128, activation=middle_activation)(out)
    out = Dense(64, activation=middle_activation)(out)
    out = Dense(32, activation='sigmoid')(out)

    #Capa final de clasificación
    if is_classification_dataset:
        if n_classes == 2:
            out = Dense(1, activation='sigmoid')(out)
        else:
            out = Dense(n_classes, activation='softmax')(out)
    else:
        out = Dense(1, activation="linear")(out)

    model = Model(input_shape, out)

    # Compile the model
    model.compile(
        optimizer = "adam",
        metrics = METRICS_REGRESSION if not is_classification_dataset else \
            (METRICS_CLASSIFICATION_BINARY if n_classes == 2 else METRICS_CLASSIFICATION_MULTICLASS),
        loss = "mean_squared_error" if not is_classification_dataset else \
            ("binary_crossentropy" if n_classes == 2 else "categorical_crossentropy")
    )

    return model

In [ ]:
def train_model(
    model,
    X_train,
    X_val,
    y_train,
    y_val,
    batch_size,
    classes_weight
):
    model.fit(
        x = X_train,
        y = y_train,
        validation_data = (X_val, y_val),
        batch_size = batch_size,
        epochs = EPOCHS,
        callbacks = [
            tf.keras.callbacks.EarlyStopping(
                monitor = 'val_loss',
                patience = 20,
                restore_best_weights = True,
                start_from_epoch = 10
            )
        ],
        class_weight = classes_weight
    )

In [ ]:
def get_partial_load_image(image_method_folder):
    def load_image(img_path):
        return cv2.imread(
            os.path.join(image_method_folder, img_path)
        )
    
    return load_image

In [ ]:
def train_with_dataset(dataset_name):
    dataset_folder = utils.get_cnnmodels_path(dataset_name)

    # Get all available image methods
    available_images = {
        image_method
        for image_method in utils.ALL_IMAGE_METHODS
        if any(
            f.endswith("csv") for f in os.listdir(utils.get_images_path_for_dataset(dataset_name, image_method))
        )
    }
    
    # Get all already done image methods
    finished_methods = set([f.split("_")[0] for f in os.listdir(dataset_folder)])

    # Get all remaining methods
    remaining_methods = available_images - finished_methods

    if not remaining_methods:
        return

    # Get the dataset
    X,y = utils.get_X_y(dataset_name)
    del X   # In this experiment I use images (not the raw data)

    # Get the indices for train and validation split
    indices_train,indices_val = utils.get_indices_train_eval(dataset_name)
    y_train = y[indices_train]
    y_val = y[indices_val]

    # Get the class weights
    classes = np.unique(y_train)
    class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train)
    equal_weights = compute_class_weight(class_weight=None, classes=classes, y=y_train)

    class_weight_dict = {classes[i]: weight for i, weight in enumerate(class_weights)}
    equal_weight_dict = {classes[i]: weight for i, weight in enumerate(equal_weights)}

    if utils.is_dataset_classification(dataset_name):
        is_classification_dataset = True
        n_classes = utils.get_number_of_classes(dataset_name)
        assert n_classes > 1
    else:
        is_classification_dataset = False
        n_classes = None

    if utils.is_dataset_multiclass_classification(dataset_name):
        y_train = to_categorical(y_train, num_classes=n_classes)
        y_val = to_categorical(y_val, num_classes=n_classes)
    del y

    for image_method in sorted(remaining_methods):
        image_method_folder = utils.get_images_path_for_dataset(dataset_name, image_method)

        # Load the routes to the images
        csv_file_path = os.path.join(
            image_method_folder,
            next(f for f in os.listdir(image_method_folder) if f.endswith(".csv"))
        )
        image_paths = pd.read_csv(csv_file_path)

        func_load_image = get_partial_load_image(image_method_folder)

        image_paths_np = image_paths["images"].to_numpy()
        train_paths = image_paths_np[indices_train]
        val_paths = image_paths_np[indices_val]

        del image_paths

        # Load train and validation images
        with concurrent.futures.ThreadPoolExecutor() as executor:
            X_train = np.array(list(executor.map(func_load_image, train_paths)))
            X_val = np.array(list(executor.map(func_load_image, val_paths)))

        for dropout_value in DROPOUT_VALUES:
            for middle_activation in MIDDLE_ACTIVATIONS:
                for balancing_classes in [False, True]:
                    for last_layer_n in [128, 64]:
                        print(image_method, dropout_value, middle_activation, balancing_classes)

                        # Build the model
                        model = build_model(
                            input_shape = X_train[0].shape,
                            is_classification_dataset = is_classification_dataset,
                            dropout_value = dropout_value,
                            middle_activation = middle_activation,
                            last_layer_n = last_layer_n,
                            n_classes = n_classes,
                        )
                        
                        # Train the model
                        train_model(
                            model = model,
                            X_train = X_train,
                            X_val = X_val,
                            y_train = y_train,
                            y_val = y_val,
                            batch_size = BATCH_SIZES[dataset_name],
                            classes_weight = class_weight_dict if balancing_classes else equal_weight_dict
                        )

                        # Save the model
                        model.save(os.path.join(dataset_folder, f"{image_method}_{middle_activation}_{dropout_value}_balance={balancing_classes}.keras"))

## HELOC

In [ ]:
train_with_dataset(HELOC_NAME)

## DENGUE

In [ ]:
train_with_dataset(DENGUE_DATASET)

## California Housing

In [ ]:
train_with_dataset(CALIFORNIA_HOUSING_NAME)

## Covertype

In [ ]:
train_with_dataset(COVERTYPE_NAME)